**Steps to run:**    
1. Use google collab
2. Make sure the data set and data loader are uploaded as files on Collab.


In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
np.random.seed(0)
import data_loader
import pandas as pd

import keras
from keras.models import Sequential
from keras.layers import AveragePooling2D, Input, Dense, Flatten
from keras.optimizers import SGD

from keras.layers import Dropout, Activation, Conv2D, MaxPooling2D, MaxPooling1D,GlobalAveragePooling1D,GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization 
from keras import regularizers 
from keras.layers.convolutional import Conv1D
from keras.layers.core import Dense, Dropout, Flatten
from keras.applications.resnet50 import ResNet50
from keras.models import Model


Using TensorFlow backend.


####Image loading and Pre-processing

In [0]:
train_files ,train_labels ,valid_files, valid_labels, test_files ,test_labels, label_dict = \
   data_loader.load_dataset('./project/images', 'project/data/legend.csv', valid_rate=0.1)

In [0]:
!unzip -uq "/content/facial_expressions.zip"

In [0]:
def image_to_feature_vector(image, size=(64, 64)):
	return cv2.resize(image, size)#.flatten()

In [0]:
tr_images = []
ts_images = []
va_images = []

for t,im_path in enumerate(train_files):
  img = cv2.imread(im_path, 1)
  img2 = image_to_feature_vector(img)
  tr_images.append(img2)

for t,im_path in enumerate(test_files):
  img = cv2.imread(im_path, 1)
  img2 = image_to_feature_vector(img)
  ts_images.append(img2)

for t,im_path in enumerate(valid_files):
  img = cv2.imread(im_path, 1)
  img2 = image_to_feature_vector(img)
  va_images.append(img2)

In [7]:
tr_images = np.array(tr_images)
ts_images = np.array(ts_images)
va_images = np.array(va_images)

print(tr_images.shape)

(11086, 64, 64, 3)


In [0]:
n_classes = 8
y_train = keras.utils.to_categorical(train_labels, n_classes)
y_test = keras.utils.to_categorical(test_labels, n_classes)
y_valid = keras.utils.to_categorical(valid_labels, n_classes)

In [9]:
print(y_train.shape)
print(y_test.shape)
print(y_valid.shape)

tri = np.expand_dims(tr_images, axis=3)
tsi = np.expand_dims(ts_images, axis=3)
vai = np.expand_dims(va_images, axis=3)

print(tri.shape)
print(ts_images.shape)
print(va_images.shape)

(11086, 8)
(1378, 8)
(1226, 8)
(11086, 64, 64, 1, 3)
(1378, 64, 64, 3)
(1226, 64, 64, 3)


####VGG Model:

1. Tried VGG model
2. Stopped execution as results/accuracies were not high as expected
3. Not included in report

In [0]:
def create_model():
  model = Sequential()
  # input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
  # this applies 32 convolution filters of size 3x3 each.
  model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.1))

  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.1))

  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(8, activation='softmax'))

  sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

In [0]:
vgg = create_model()

In [0]:
vgg.fit(tri, y_train, batch_size=32, epochs=10)

Epoch 1/10
11086/11086 [==============================] - 633s 57ms/step - loss: 8.0414 - acc: 0.5005
Epoch 2/10
 4800/11086 [===========>..................] - ETA: 5:44 - loss: 7.9885 - acc: 0.5044

KeyboardInterrupt: ignored

## ResNet with pretrained weights

One can observe faster training time. Accuracy is low and comparable with k-NN

In [0]:
def create_Res():
  base_model = ResNet50(weights= 'imagenet', include_top=False, input_shape= (64,64,3))
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dropout(0.1)(x)
  predictions = Dense(8, activation= 'softmax')(x)
  model = Model(inputs = base_model.input, outputs = predictions)
#   x res.layers.pop()
#   res.layers.Add(Dense(8, activation='softmax'))
  l = len(model.layers)-1
  for i in range(1,l):
    model.layers[i].trainable = False
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()
  return model

In [0]:
res = create_Res()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_17[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
res.fit(tr_images, y_train, batch_size=32, epochs=50)

Epoch 1/50
11086/11086 [==============================] - 184s 17ms/step - loss: 1.1705 - acc: 0.5895
Epoch 2/50
11086/11086 [==============================] - 138s 12ms/step - loss: 0.9401 - acc: 0.6493
Epoch 3/50
11086/11086 [==============================] - 139s 13ms/step - loss: 0.8725 - acc: 0.6594
Epoch 4/50
11086/11086 [==============================] - 138s 12ms/step - loss: 0.8532 - acc: 0.6620
Epoch 5/50
11086/11086 [==============================] - 138s 12ms/step - loss: 0.8379 - acc: 0.6711
Epoch 6/50
11086/11086 [==============================] - 138s 12ms/step - loss: 0.8291 - acc: 0.6686
Epoch 7/50
11086/11086 [==============================] - 138s 12ms/step - loss: 0.8175 - acc: 0.6719
Epoch 8/50
11086/11086 [==============================] - 138s 12ms/step - loss: 0.8191 - acc: 0.6713
Epoch 9/50
11086/11086 [==============================] - 138s 12ms/step - loss: 0.7985 - acc: 0.6786
Epoch 10/50
11086/11086 [==============================] - 138s 12ms/step - loss: 

## ResNet with Retraining the entire model. 

Training time increased dramatically and Highest accuracy is observed (97% for training and 82% for test and validation.)

In [0]:
def create_Res2():
  base_model = ResNet50(include_top=False, input_shape= (64,64,3))
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dropout(0.1)(x)
  predictions = Dense(8, activation= 'softmax')(x)
  model = Model(inputs = base_model.input, outputs = predictions)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()
  return model

In [11]:
res2 = create_Res2()

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 8s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization

In [13]:
res2.fit(tr_images, y_train, batch_size=32, epochs=30)

Epoch 1/30
11086/11086 [==============================] - 892s 80ms/step - loss: 1.0877 - acc: 0.7182
Epoch 2/30
11086/11086 [==============================] - 894s 81ms/step - loss: 0.8569 - acc: 0.7906
Epoch 3/30
11086/11086 [==============================] - 893s 81ms/step - loss: 0.8763 - acc: 0.7865
Epoch 4/30
11086/11086 [==============================] - 902s 81ms/step - loss: 0.7901 - acc: 0.7946
Epoch 5/30
11086/11086 [==============================] - 899s 81ms/step - loss: 0.9099 - acc: 0.7819
Epoch 6/30
11086/11086 [==============================] - 902s 81ms/step - loss: 0.9439 - acc: 0.7850
Epoch 7/30
11086/11086 [==============================] - 904s 82ms/step - loss: 0.8278 - acc: 0.7803
Epoch 8/30
11086/11086 [==============================] - 901s 81ms/step - loss: 0.5762 - acc: 0.8125
Epoch 9/30
11086/11086 [==============================] - 903s 81ms/step - loss: 0.4700 - acc: 0.8405
Epoch 10/30
11086/11086 [==============================] - 900s 81ms/step - loss: 

In [24]:
val_score = res2.evaluate(va_images, y_valid, batch_size=32)
test_score = res2.evaluate(ts_images, y_test, batch_size=32)

1378/1378 [==============================] - 18s 13ms/step


In [25]:
print("test score", res2.metrics_names, test_score)
print("validation score", res2.metrics_names, val_score)

test score ['loss', 'acc'] [0.9122135738225222, 0.8200290275761973]
validation score ['loss', 'acc'] [0.8667214865785633, 0.8205546492902139]
